In [41]:
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
df = pd.read_excel("Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


Lable 별 의미
InvoiceNo: 거래 번호, C로 시작시 최소된 거래
StockCode: 제품 번호
Description: 제품이름
Quantity: 거래당 각 제품의 수량
InvoiceDate: 각거래가 생성된 날짜와 시간
UnitPrice: 단위당 제품 가격 즉 단가
CustomerID: 고객 번호
Country: 구매한 고객이 거주하는 국가

In [12]:
print(df.isnull().sum())
print(df["InvoiceNo"].nunique())
print(df["StockCode"].nunique())
print(df["Description"].nunique())
print(df["InvoiceDate"].nunique())
print(df["UnitPrice"].nunique())
print(df["CustomerID"].nunique())
print(df["Country"].nunique())


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64
22190
3684
3896
20460
620
4372
37


실수형 타겟이 될 수 있는 TotalPrice를 만들기위해 Lable 생성 및 결측치 제거, 취소된 거래인 C가 들어간 거래 제거

In [19]:
df.dropna(inplace=True)
df.describe().T
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
df.info()
df = df[~df["InvoiceNo"].str.contains("C", na = False)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
 8   TotalPrice   406829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.0+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   Sto

모델 RandomforestRegrssion을 만들려면 타겟은 실수형인 Totalprice로 설정 그 후 Unitprice와 Quantity를 제거후 고유번호인 InvoiceNo를 제거한다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
X = df.drop(columns=['InvoiceNo', 'Country'])
y = df['Country']
X_encoded = X.copy()

if 'InvoiceDate' in X_encoded.columns:
    X_encoded['Year'] = X_encoded['InvoiceDate'].dt.year
    X_encoded['Month'] = X_encoded['InvoiceDate'].dt.month
    X_encoded['Day'] = X_encoded['InvoiceDate'].dt.day
    X_encoded = X_encoded.drop(columns=['InvoiceDate'])

for col in X_encoded.columns:
    if X_encoded[col].dtype == 'object':
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2,random_state=42)

model_RFR = RandomForestClassifier(random_state=42, n_jobs=-1)
model_RFR.fit(X_train,y_train)
predict_RFR = model_RFR.predict(X_test)


In [43]:
print("Accuracy:", accuracy_score(y_test, predict_RFR))
print("F1 (macro):", f1_score(y_test, predict_RFR, average='macro'))
print("Precision (macro):", precision_score(y_test, predict_RFR, average='macro'))
print("Recall (macro):", recall_score(y_test, predict_RFR, average='macro'))

Accuracy: 0.9956273167054093
F1 (macro): 0.9502214726918187
Precision (macro): 0.9873425225870708
Recall (macro): 0.9246985106410139


In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict_RFR))

                      precision    recall  f1-score   support

           Australia       0.99      0.98      0.99       241
             Austria       0.99      0.95      0.97        73
             Bahrain       1.00      1.00      1.00         4
             Belgium       0.97      0.99      0.98       398
              Brazil       1.00      0.75      0.86         4
              Canada       1.00      0.79      0.89        34
     Channel Islands       1.00      0.94      0.97       154
              Cyprus       0.99      0.95      0.97       115
      Czech Republic       1.00      1.00      1.00         4
             Denmark       0.95      0.91      0.93        76
                EIRE       1.00      0.98      0.99      1438
  European Community       1.00      1.00      1.00         6
             Finland       0.96      0.90      0.93       153
              France       0.97      0.97      0.97      1653
             Germany       0.97      0.97      0.97      1855
       